# 지도학습 1. 분류 (Classification)
## 데이터 로드
- 과일 데이터(fruits_300.npy)를 로드합니다.
- 타깃 값을 만듭니다(fruits_target).
- 훈련 / 테스트 셋을 분할합니다. (random_state=42, 타깃 라벨 비율을 유지하며 분할, 다른 설정값은 디폴트 지정)

In [80]:
import numpy as np

fruits = np.load("fruits_300.npy")
fruits_2d = fruits.reshape(-1, 100*100)

In [81]:
fruits_target = ['apple']*100 + ['pineapple']*100 + ['banana']*100

In [82]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    fruits_2d, fruits_target, random_state=42, stratify=fruits_target)

## 분류기 학습 및 성능 비교
- 대상 모델: KNN, 로지스틱 회귀
- 성능 지표: 정확도
- 훈련, 테스트 셋 성능 비교
- 파라미터 디폴트 값으로 설정

In [83]:
from sklearn.neighbors import KNeighborsClassifier

kn = KNeighborsClassifier()
kn.fit(train_input, train_target)
print(kn.score(train_input, train_target))
print(kn.score(test_input, test_target))

0.9955555555555555
1.0


In [84]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(train_input, train_target)
print(lr.score(train_input, train_target))
print(lr.score(test_input, test_target))

1.0
1.0


## 분류 모델 실험 결과에 대해 고찰합니다.


# 지도학습 2. 회귀 (Regression)
## 데이터 로드 및 전처리
- 판다스 함수를 이용해 광고비-매출 데이터(advertising.csv)를 로드합니다.
- 입력 특성(TV, Radio, Newspaper)과 타겟(Sales)을 분리합니다.
- 훈련 / 테스트 셋을 분할합니다. (random_state=42, 다른 설정값은 디폴트 지정)

In [85]:
import pandas as pd

advertising = pd.read_csv('advertising.csv')

In [86]:
advertising_input = advertising.drop(columns='Sales')
advertising_target = advertising['Sales']

In [87]:
train_input, test_input, train_target, test_target = train_test_split(
    advertising_input, advertising_target, random_state=42)

## 회귀 모델 학습 및 성능 비교
- 대상 모델: 다중 선형 회귀, 다항 회귀(최대항 3), 다항 회귀(최대항 4)
- 성능 지표: 결정계수(R²)
- 훈련, 테스트 셋 성능 비교
- 모델 설정값: 다항 회귀의 include_bias=False 지정, 나머지는 디폴트 지정
- 전처리: StandardScaler 적용

In [88]:
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

scaler = StandardScaler()
train_scaled = scaler.fit_transform(train_input)
test_scaled = scaler.transform(test_input)

linear_model = LinearRegression()
linear_model.fit(train_scaled, train_target)

train_pred_linear = linear_model.predict(train_scaled)
test_pred_linear = linear_model.predict(test_scaled)

r2_train_linear = r2_score(train_target, train_pred_linear)
r2_test_linear = r2_score(test_target, test_pred_linear)

print("다중 선형 회귀:")
print(f"훈련 세트 R²: {r2_train_linear:.4f}")
print(f"테스트 세트 R²: {r2_test_linear:.4f}")

다중 선형 회귀:
훈련 세트 R²: 0.8958
테스트 세트 R²: 0.9116


In [89]:
def evaluate_polynomial(degree):
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    train_poly = poly.fit_transform(train_scaled)
    test_poly = poly.transform(test_scaled)
    
    poly_model = LinearRegression()
    poly_model.fit(train_poly, train_target)
    
    train_pred = poly_model.predict(train_poly)
    test_pred = poly_model.predict(test_poly)
    
    r2_train = r2_score(train_target, train_pred)
    r2_test = r2_score(test_target, test_pred)
    
    print(f"\n다항 회귀 (최대항 {degree}):")
    print(f"훈련 세트 R²: {r2_train:.4f}")
    print(f"테스트 세트 R²: {r2_test:.4f}")
    
evaluate_polynomial(degree=3)
evaluate_polynomial(degree=4)


다항 회귀 (최대항 3):
훈련 세트 R²: 0.9364
테스트 세트 R²: 0.9362

다항 회귀 (최대항 4):
훈련 세트 R²: 0.9545
테스트 세트 R²: 0.6179


## 회귀 모델 규제
- Ridge, Lasso 를 이용해 다항 회귀 최대항 4 모델을 규제합니다.
- 규제 모델의 입력은 Z-score 로 전처리 합니다.
- 디폴트 설정값을 사용합니다.

In [90]:
from sklearn.linear_model import Ridge, Lasso

poly = PolynomialFeatures(degree=4, include_bias=False)
train_poly = poly.fit_transform(train_scaled)
test_poly = poly.transform(test_scaled)

In [91]:
ridge = Ridge()
ridge.fit(train_poly, train_target)

ridge_train_pred = ridge.predict(train_poly)
ridge_test_pred = ridge.predict(test_poly)

ridge_r2_train = r2_score(train_target, ridge_train_pred)
ridge_r2_test = r2_score(test_target, ridge_test_pred)

print("Ridge 회귀:")
print(f"훈련 세트 R²: {ridge_r2_train:.4f}")
print(f"테스트 세트 R²: {ridge_r2_test:.4f}")

Ridge 회귀:
훈련 세트 R²: 0.9541
테스트 세트 R²: 0.6155


In [92]:
lasso = Lasso()
lasso.fit(train_poly, train_target)

lasso_train_pred = lasso.predict(train_poly)
lasso_test_pred = lasso.predict(test_poly)

lasso_r2_train = r2_score(train_target, lasso_train_pred)
lasso_r2_test = r2_score(test_target, lasso_test_pred)

print("\nLasso 회귀:")
print(f"훈련 세트 R²: {lasso_r2_train:.4f}")
print(f"테스트 세트 R²: {lasso_r2_test:.4f}")


Lasso 회귀:
훈련 세트 R²: 0.8319
테스트 세트 R²: 0.7909


## 회귀 모델 실험 결과에 대해 고찰합니다.